In [169]:
import pandas as pd
import zipfile
import numpy as np

### A Fazeres
- Importar dados do IPCA <br>
Precisa converter datas em datetime
- Adicionar dados trimestrais e todos os indicadores imobiliários disponíveis neles

- Tabelas descritivas dos dados, gráficos, etc...
- Descrever as variáveis disponíveis nos dados e destacar as mais importantes



Referência:<br>
Dados FIIs - https://dados.cvm.gov.br/dataset/fii-doc-inf_mensal<br>
IPCA - https://www.ibge.gov.br/estatisticas/economicas/precos-e-custos/9256-indice-nacional-de-precos-ao-consumidor-amplo.html?=&t=series-historicas<br>
CDI - https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-de-segmentos-e-setoriais/serie-historica-do-di.htm <br>
Indice de negociabilidade - Economática<br>
Benchmarks - Economatica<br>
Fechamentos - Economatica<br>
Características Fiis - CVM e Economática<br>


### Problemas:

- Preços estam problemáticos / Total Return problemático
- Dados de fechamento não coincidem com realizado, mesmo se tratando dos preços sem ajuste de proventos
- Base CNPJ/Tickers tem dados faltantes

Importando os dados FIIs

In [87]:
start_date = 2016
end_date = 2024

diretorio = r"C:\Users\Gustavo Aurelio\Desktop\TCC\clone_repositorio_fii\fiis_project\bases_zip"
base_mensal = pd.DataFrame()

for year in range(start_date, end_date+1):
    zip_mensal = zipfile.ZipFile(diretorio+ f"\Mensal_zip\inf_mensal_fii_{year}.zip")
    # zip_trimestral = zipfile.ZipFile(diretorio + f"\Trimestral_zip\inf_trimestral_fii_{year}.zip")

    bp_inf_mensal = pd.read_csv(zip_mensal.open(f'inf_mensal_fii_ativo_passivo_{year}.csv'), delimiter=";",encoding='latin-1')
    complem_inf_mensal = pd.read_csv(zip_mensal.open(f'inf_mensal_fii_complemento_{year}.csv'), delimiter=";", encoding='latin-1')
    geral_inf_mensal = pd.read_csv(zip_mensal.open(f'inf_mensal_fii_geral_{year}.csv'), delimiter=";", encoding='latin-1')
    base_mensal_year=bp_inf_mensal.merge(complem_inf_mensal,how='outer',on=['Data_Referencia','CNPJ_Fundo','Versao'])
    base_mensal_year=base_mensal_year.merge(geral_inf_mensal,how='outer',on=['Data_Referencia','CNPJ_Fundo','Versao'])

    base_mensal = pd.concat([base_mensal, base_mensal_year], ignore_index=True)

    #Precisa ajustar base trimestral
    # imovel_inf_trimestral = pd.read_csv(zip_trimestral.open(f'inf_trimestral_fii_imovel_{year}.csv'), delimiter=";",encoding='latin-1')
    # result_inf_trimestral = pd.read_csv(zip_trimestral.open(f'inf_trimestral_fii_resultado_contabil_financeiro_{year}.csv'), delimiter=";", encoding='latin-1')

base_mensal['Data_Referencia'] = pd.to_datetime(base_mensal.Data_Referencia)
cdi = pd.read_csv(r'C:\Users\Gustavo Aurelio\Desktop\TCC\bases_gerais\cdi.csv', delimiter=";", header = 38 )
ipca = pd.read_csv(r'C:\Users\Gustavo Aurelio\Desktop\TCC\bases_gerais\ipca.csv', delimiter=";", header = 1 )
ineg = pd.read_csv(r'C:\Users\Gustavo Aurelio\Desktop\TCC\bases_gerais\ineg_fiis.csv', delimiter = ';',header = 3)
benchmarks = pd.read_csv(r'C:\Users\Gustavo Aurelio\Desktop\TCC\bases_gerais\economatica_benchmarks.csv', delimiter=";", header = 3 )
cnpj_ticker = pd.read_excel(r'C:\Users\Gustavo Aurelio\Documents\repositorio\Projeto FIIs\base\CNPJ e TICKER FIIs.xlsx',sheet_name='FIIs',header=6)
fech_nao_ajust = pd.read_csv(r'C:\Users\Gustavo Aurelio\Desktop\TCC\bases_gerais\economatica_fech_nao_ajust.csv',delimiter=';',header=3)
fech_ajust_ed = pd.read_excel(r'C:\Users\Gustavo Aurelio\Desktop\TCC\bases_gerais\economatica_fech_ajust_sem_divid.xlsx',header=3)
fech_divid = pd.read_excel(r'C:\Users\Gustavo Aurelio\Desktop\TCC\bases_gerais\economatica_fech_ajust_com_div.xlsx',header=3)

In [138]:
retorno_total = fech_divid.pct_change()
retorno_total

Importando dados auxiliares

Cleaning and adjusting database

In [70]:
#Cleaning no ajustments close price
fech_nao_ajust.columns = fech_nao_ajust.columns.str.replace('Fechamento\nnão aj p/ prov\nEm moeda orig\n','',regex = True)
fech_nao_ajust = fech_nao_ajust.replace({',':'.',
                        '-':np.nan,
                        'Fechamento\nnão aj p/ prov\nEm moeda orig\n':''},regex=True)
fech_nao_ajust['Data'] = pd.to_datetime(fech_nao_ajust.Data,dayfirst=True)

datas_desde_2003 = pd.date_range(start='2003-01-01', end=pd.Timestamp.today(), freq='D')
df_datas = pd.DataFrame({'Data': datas_desde_2003})
fech_nao_ajust = pd.merge(df_datas, fech_nao_ajust, on='Data', how='left')
fech_nao_ajust = fech_nao_ajust.ffill()

fech_nao_ajust = fech_nao_ajust.set_index('Data')
fech_nao_ajust = fech_nao_ajust.apply(pd.to_numeric, errors='ignore').reset_index()
fech_nao_ajust = fech_nao_ajust.ffill() #preencher valores faltantes com ultimo valor disponível, nos casos de final de semana ou feriados terem dados de referência

fech_tidy = fech_nao_ajust.melt(id_vars='Data')
fech_tidy.columns = ['Data_Referencia', 'Ticker', 'Fechamento']

#Calculo do dividendo
fech_ajust_ed.columns = fech_ajust_ed.columns.str.replace('Fechamento\najus p/prov ed\nEm moeda orig\n','',regex = True)
fech_ajust_ed = fech_ajust_ed.replace({',':'.',
                        '-':np.nan},regex=True)
fech_ajust_ed['Data'] = pd.to_datetime(fech_ajust_ed.Data,dayfirst=True)
fech_ajust_ed = fech_ajust_ed.set_index('Data')


fech_divid.columns = fech_divid.columns.str.replace('Fechamento\najust p/ prov\nEm moeda orig\n','',regex = True)
fech_divid = fech_divid.replace({',':'.',
                        '-':np.nan
                        },regex=True)
fech_divid['Data'] = pd.to_datetime(fech_divid.Data,dayfirst=True)
fech_divid = fech_divid.set_index('Data')


#Cleaning monthly database
base_mensal['CNPJ_Fundo'] = base_mensal['CNPJ_Fundo'].str.replace('/','',regex=False)
base_mensal['CNPJ_Fundo'] = base_mensal['CNPJ_Fundo'].str.replace('.','',regex=False)
base_mensal['CNPJ_Fundo'] = base_mensal['CNPJ_Fundo'].str.replace('-','',regex=False)

#Pareando CNPJ e Tickers
cnpj_ticker = (cnpj_ticker.loc[:,['Ticker','CNPJ/Fundo']].drop_duplicates()).dropna()
cnpj_ticker_dic = dict(zip(cnpj_ticker['CNPJ/Fundo'], cnpj_ticker['Ticker']))
base_mensal['Ticker'] = base_mensal['CNPJ_Fundo'].map(cnpj_ticker_dic)

cdi = cdi.replace({',':'.',
                    '-':np.nan,
                    'ND':np.nan},regex=True)
cdi['Taxa SELIC'] = pd.to_numeric(cdi['Taxa SELIC'])
cdi['Data'] = pd.to_datetime(cdi.Data, dayfirst=True)
cdi.columns = ['Data', 'tx_selic']


benchmarks.columns = benchmarks.columns.str.replace('Fechamento\najust p/ prov\nEm moeda orig\n','', regex = True)
benchmarks = benchmarks.replace({',':'.',
                        '-':np.nan},regex=True)
benchmarks['Data'] = pd.to_datetime(benchmarks.Data, dayfirst=True)


ineg.columns = ineg.columns.str.replace('Negociabilidade\n1 meses\nEm moeda orig\n','',regex = True)
ineg = ineg.replace({',':'.',
                        '-':np.nan
                        },regex=True)
ineg['Data'] = pd.to_datetime(ineg.Data,dayfirst=True)
ineg = ineg.set_index('Data')

In [83]:
indicadores = pd.DataFrame()
indicadores['CNPJ_Fundo'] = base_mensal['CNPJ_Fundo']
indicadores['Data_Referencia'] = base_mensal['Data_Referencia']
indicadores['Versao'] = base_mensal['Versao']
indicadores['Ticker'] = base_mensal['Ticker']
indicadores = indicadores.merge(fech_tidy,left_on = ['Ticker','Data_Referencia'], right_on = ['Ticker', 'Data_Referencia'], how= 'left')
indicadores['Patrimonio_liquido'] = base_mensal.Patrimonio_Liquido
indicadores['Total_Ativo'] = base_mensal.Valor_Ativo
indicadores['Total_Passivo'] = base_mensal.Total_Passivo
indicadores['Disponibilidades'] = base_mensal.Disponibilidades
indicadores['Rend_distrib'] = base_mensal.Rendimentos_Distribuir
indicadores['n_cotas'] = base_mensal.Cotas_Emitidas
indicadores['Valor_patri_cotas'] = base_mensal.Valor_Patrimonial_Cotas
indicadores['Div_p_cota'] = indicadores.Rend_distrib/indicadores.n_cotas

indicadores['Valor_cx_perc'] = indicadores.Disponibilidades/indicadores.Total_Ativo
indicadores['Alavancagem'] = indicadores.Total_Passivo/indicadores.Patrimonio_liquido
indicadores['P_vp'] = indicadores.Fechamento/indicadores.Valor_patri_cotas
indicadores['Div_yeld'] = indicadores.Div_p_cota/indicadores.Fechamento
indicadores['conferir_dymes'] = base_mensal.Percentual_Dividend_Yield_Mes

#Dividendos 12 Meses
Dividendos = indicadores[['Data_Referencia','Ticker','Rend_distrib']].set_index('Data_Referencia')
Dividendos = Dividendos.pivot_table(index='Data_Referencia',columns='Ticker',values='Rend_distrib')
Dividendos_12 = Dividendos.rolling(12).sum().reset_index().melt(id_vars='Data_Referencia')
Dividendos_12.columns = ['Data_Referencia', 'Ticker', 'Dividendos_12']
indicadores = indicadores.merge(Dividendos_12,left_on = ['Ticker','Data_Referencia'], right_on = ['Ticker', 'Data_Referencia'], how= 'left')

indicadores['Dividend_yield_12'] = (indicadores.Dividendos_12/indicadores.n_cotas)/indicadores.Fechamento

In [166]:
#Teste
len(indicadores.CNPJ_Fundo.unique())

1099

In [167]:
len(indicadores.Ticker.unique())

358

Adicionar Dados Trimestrais

P/M²

Aluguel/M²

PL/M²